Converting Pdfs to images of front and back for processing

In [8]:
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import os

# Assuming the PDF file is named "sample.pdf" and located in the current directory
pdf_path = "Pdfs/1771240124_Double Sided.pdf"
output_folder = "Images/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Extracting images using pdf2image
images = convert_from_path(pdf_path)

# Create folders and save images
form_counter = 1
for i in range(0, len(images), 2):  # Assuming each form has a front and back image
    form_folder = os.path.join(output_folder, f"form_{form_counter}")
    if not os.path.exists(form_folder):
        os.makedirs(form_folder)
    
    front_image_path = os.path.join(form_folder, f"form_{form_counter}_front.jpg")
    back_image_path = os.path.join(form_folder, f"form_{form_counter}_back.jpg")
    
    images[i].save(front_image_path, "JPEG")
    if i+1 < len(images):
        images[i+1].save(back_image_path, "JPEG")
    
    form_counter += 1

result = f"Extracted and saved images for {form_counter-1} forms."
result


'Extracted and saved images for 11 forms.'

Creating masking funtion for front and back.

In [9]:
import cv2
import numpy as np
import os

def masked_image(image_path, rect_params, question_name, form_number):
    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        raise FileNotFoundError(f"The file at {image_path} does not exist.")

    # Unpack rectangle parameters
    rect_x, rect_y, rect_width, rect_height = rect_params

    # Create a mask with the same dimensions as the image, initialized to black
    mask = np.zeros(image.shape[:2], dtype="uint8")

    # Draw a white rectangle on the mask
    cv2.rectangle(mask, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), 255, -1)

    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, image, mask=mask)

    output_dir = f"Images/form_{form_number}/{question_name}"
    output_path = f"{output_dir}/{question_name}.jpg"

    # Check if the output directory exists, create it if not
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the result to a file
    cv2.imwrite(output_path, masked_image)
    pass


def process_front_side(image_path,form_number):
    sections = [
        ([285, 200, 1500, 80], "Title"),
        ([70, 330, 1500, 200], "Q1"),
        ([70, 605, 1500, 140], "Q2"),
        ([70, 778, 1500, 140], "Q3"),
        ([70, 980, 1500, 140], "Q4")
        
    ]
    
    for rect_params, label in sections:
        masked_image(image_path, rect_params, label, form_number)

def process_back_side(image_path,form_number):
    sections = [
        ([100, 120, 1500, 140], "Q5"),
        ([100, 320, 1500, 140], "Q6"),
        ([100, 530, 1500, 130], "Q7"),
        ([330, 660, 800, 80], "Name"),
        ([330, 730, 1100, 80], "School"),
        ([1305, 834, 22, 16], "CheckBox")
    ]
    
    for rect_params, label in sections:
        masked_image(image_path, rect_params, label, form_number)


In [10]:
for i in range(1, 12):
    front_image_path = f'Images/form_{i}/form_{i}_front.jpg'
    process_front_side(front_image_path,i)

    back_image_path = f'Images/form_{i}/form_{i}_back.jpg'
    process_back_side(back_image_path,i)
    



Display Images Code

In [11]:
import cv2
import numpy as np
import os
from IPython.display import Image, display, HTML


def apply_mask(image, mask):
    # Apply the mask to the image
    return cv2.bitwise_and(image, image, mask=mask)

def save_image(image, output_path):
    # Check if the output directory exists, create it if not
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the result to a file
    cv2.imwrite(output_path, image)

def create_cumulative_mask(image_shape, sections):
    # Create a cumulative mask with the same dimensions as the image, initialized to black
    cumulative_mask = np.zeros(image_shape[:2], dtype="uint8")

    # Draw white rectangles for all sections on the cumulative mask
    for rect_params in sections:
        rect_x, rect_y, rect_width, rect_height = rect_params
        cv2.rectangle(cumulative_mask, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), 255, -1)
    
    return cumulative_mask

def process_image(image_path, form_number, front=True):
    sections = [
        ([285, 200, 1300, 80], "Title"),
        ([70, 330, 1500, 200], "Q1"),
        ([70, 605, 1500, 140], "Q2"),
        ([70, 778, 1500, 140], "Q3"),
        ([70, 980, 1500, 140], "Q4"),
        ([1305, 834, 22, 16], "CheckBox")
    ] if front else [
        ([100, 120, 1500, 140], "Q5"),
        ([100, 320, 1500, 140], "Q6"),
        ([100, 530, 1500, 130], "Q7"),
        ([330, 660, 800, 80], "Name"),
        ([330, 730, 1200, 80], "School")
    ]

    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"The file at {image_path} does not exist.")

    # Create cumulative mask for all sections
    cumulative_mask = create_cumulative_mask(image.shape, [rect_params for rect_params, _ in sections])

    # Apply cumulative mask to show all sections
    masked_image_all_sections = apply_mask(image, cumulative_mask)
    save_image(masked_image_all_sections, f"Images/form_{form_number}/{'front' if front else 'back'}_all_sections.jpg")

    # Invert mask to show the inverse
    inverse_mask = cv2.bitwise_not(cumulative_mask)
    masked_image_inverse = apply_mask(image, inverse_mask)
    save_image(masked_image_inverse, f"Images/form_{form_number}/{'front' if front else 'back'}_inverse.jpg")

# Example usage
process_image("Images/form_3/form_3_front.jpg", form_number=3, front=True) # For front side


# Define the paths to your images
image_path1 = 'Images/form_3/front_inverse.jpg'
image_path2 = 'Images/form_3/front_all_sections.jpg'

# Create an HTML string that includes the images with desired width or height
html_str = f"""
<table>
<tr>
<td><img src='{image_path1}' width='500'/></td>
<td><img src='{image_path2}' width='500'/></td>
</tr>
</table>
"""

# Display the HTML in the notebook
display(HTML(html_str))


,


In [12]:
process_image("Images/form_1/form_1_back.jpg", form_number=1, front=False) # For back side

# Define the paths to your images
image_path1 = 'Images/form_1/back_inverse.jpg'
image_path2 = 'Images/form_1/back_all_sections.jpg'

# Create an HTML string that includes the images with desired width or height
html_str = f"""
<table>
<tr>
<td><img src='{image_path1}' width='500'/></td>
<td><img src='{image_path2}' width='500'/></td>
</tr>
</table>
"""

# Display the HTML in the notebook
display(HTML(html_str))

,


In [13]:
import requests

url = "https://pen-to-print-handwriting-ocr.p.rapidapi.com/recognize/"

files = { "srcImg": "open('Images/form_1/Name/Name.jpg', 'rb')" }
payload = { "Session": "string" }
headers = {
	"X-RapidAPI-Key": "4a7ca66e47msh44f5770ceee70f2p1fc7b0jsn8a10fdaddcbe",
	"X-RapidAPI-Host": "pen-to-print-handwriting-ocr.p.rapidapi.com"
}

response = requests.post(url, data=payload, files=files, headers=headers)

print(response.json())

{'error': "General Exception:cannot identify image file '/tmp/tmp_vsng0bq'", 'log': ['session null', 'srcUrl null', 'includeSubScan null', 'Image path:srcImg', 'Execution Time:0.23105382919311523'], 'result': '0', 'time': 0.23105382919311523}


In [14]:
from google.cloud import vision
import os

def extract_text(image_path):
    # Set the path to your service account key file
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "client_secret.json"
    
    # Create a Vision API client
    client = vision.ImageAnnotatorClient()
    
    # Load the image file
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    
    image = vision.Image(content=content)
    
    image_context = vision.ImageContext(language_hints=["en"])
    
    # Request document text detection
    response = client.document_text_detection(image=image,image_context=image_context)
    
    # Extract and return the detected text
    return response.full_text_annotation.text




In [15]:
from google.cloud import vision
import os
import cv2
import numpy as np
import io

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    # Apply thresholding
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def extract_text(image_path):
    # Set the path to your service account key file
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "client_secret.json"
    
    # Create a Vision API client
    client = vision.ImageAnnotatorClient()
    
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)
    
    # Encode the preprocessed image to bytes
    is_success, buffer = cv2.imencode(".jpg", preprocessed_image)
    if not is_success:
        raise Exception("Could not encode image to bytes")
    io_buf = io.BytesIO(buffer)
    
    # Use the bytes of the preprocessed image
    content = io_buf.read()
    
    image = vision.Image(content=content)
    image_context = vision.ImageContext(language_hints=["en"])
    
    # Request document text detection
    response = client.document_text_detection(image=image, image_context=image_context)
    
    # Extract and return the detected text
    return response.full_text_annotation.text

# Example usage
extract_text("Images/form_1/Q3/Q3.jpg")


'Nove'

In [25]:
import json

form_texts = {}
headings = ["Title", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Name", "School"]

for i in range(1, 12):
    
    for side in ["front", "back"]:
        image_path = f"Images/form_{i}/form_{i}_{side}.jpg"
        form_texts[side] = image_path
                
    for heading in headings:
        image_path = f"Images/form_{i}/{heading}/{heading}.jpg"
        form_texts[heading] = extract_text(image_path)
        with open(f'Images/form_{i}/form.json', 'w') as f:
            f.write(json.dumps(form_texts, indent=4))



In [53]:
from pyairtable import Api
import datetime

class AirtableIntegration:
    def __init__(self):
        # To avoid makin unnecessary API calls, we will store the data in a variable
        
        api = Api('patSdMngQk4qMiYtd.0045ab336889ede5642c2f2e4ebaa4294191f986d318e628c479f3a7078f4475')
        self.table = api.table('appUtp5d6xgVB3lxq', 'tblykp6tEWyGZKUKw')
        
        self.data_list = self.table.all()
        
        
    def backup(self):
        """Backup the data to a JSON file with a timestamp."""
        # Get the current date and time
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        # Create a filename with the timestamp
        filename = rf'backups\airtable_backup_{timestamp}.json'
        # Backup the data to the file
        with open(filename, 'w') as file:
            json.dump(self.data_list, file)
            
    def create_records(self,form_path):
        """Create records in the Airtable table."""
        with open(form_path, 'r') as file:
            form = json.load(file)
        
        front_image_path = form['front']
        back_image_path = form['back'] 
        
        form = self.set_correct_headings(form)
        
        self.table.create(form)

    def set_correct_headings(self,form):
        new_headings = ["Workshop Title","Q1: \ufeffOverall, what was your impression of today's workshop?","Q2: \ufeffWere there any specific strategies or techniques that stood out to you during the training? In what ways, do you believe the training event will influence your delivery/lesson planning?","Q3: \ufeffWhat areas of today's workshop require improvement? How can we ensure you attend another event with us?","Q4: What other topics or areas of professional development would you like to explore in future training events? Are there any specific needs or interests you have in mind?", "Q5: \ufeffWhat specific aspects of professional development do you consider most valuable for your career growth?","Q6: \ufeffWhat is your opinion on the concept of an online academy that offers self-directed professional development modules focused on topics such as leadership, management, classroom practice, and behaviour?","Q7: \ufeffWould academic accreditation in the form of micro-credits as part of an in-service Teacher Professional Development programme be an appealing proposition? Why? Why not?","Name","School"]


        
        form.pop('front', None)
        form.pop('back', None)
        form = {new_headings[i]: form[old_heading] for i, old_heading in enumerate(form)}
        
        return form

        
        
AT = AirtableIntegration()

# Backup the data
AT.backup()  


for i in range(1, 12):
    form_path = f'Images/form_{i}/form.json'
    AT.create_records(form_path)


In [52]:
from pyairtable import Api
import datetime

class AirtableIntegration:
    def __init__(self):
        # To avoid makin unnecessary API calls, we will store the data in a variable
        
        api = Api('patSdMngQk4qMiYtd.0045ab336889ede5642c2f2e4ebaa4294191f986d318e628c479f3a7078f4475')
        self.table = api.table('appUtp5d6xgVB3lxq', 'tblykp6tEWyGZKUKw')
        
        self.data_list = self.table.all()
        
        
    def backup(self):
        """Backup the data to a JSON file with a timestamp."""
        # Get the current date and time
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        # Create a filename with the timestamp
        filename = rf'backups\airtable_backup_{timestamp}.json'
        # Backup the data to the file
        with open(filename, 'w') as file:
            json.dump(self.data_list, file)
            
    def create_records(self, form_path):
        """Create records in the Airtable table with attachments."""
        with open(form_path, 'r') as file:
            form = json.load(file)
        
        # Directly using URLs from the form.json
        front_image_url = form['front']
        back_image_url = form['back']
        
        # Format attachments according to Airtable requirements
        attachments = [
            {"url": front_image_url},  # Assuming these are direct URLs to the images
            {"url": back_image_url}
        ]
        
        # Prepare the form data with correct headings and remove unnecessary keys
        form = self.set_correct_headings(form)
        form.pop('front', None)
        form.pop('back', None)
        
        # Include the attachments in the correct field
        # Make sure "Front Form Image" is the exact name of your attachment field in Airtable
        form["Front Form Image"] = attachments
        
        # Create the record in Airtable
        self.table.create(form)


    def set_correct_headings(self,form):
        new_headings = ["Workshop Title","Q1: \ufeffOverall, what was your impression of today's workshop?","Q2: \ufeffWere there any specific strategies or techniques that stood out to you during the training? In what ways, do you believe the training event will influence your delivery/lesson planning?","Q3: \ufeffWhat areas of today's workshop require improvement? How can we ensure you attend another event with us?","Q4: What other topics or areas of professional development would you like to explore in future training events? Are there any specific needs or interests you have in mind?", "Q5: \ufeffWhat specific aspects of professional development do you consider most valuable for your career growth?","Q6: \ufeffWhat is your opinion on the concept of an online academy that offers self-directed professional development modules focused on topics such as leadership, management, classroom practice, and behaviour?","Q7: \ufeffWould academic accreditation in the form of micro-credits as part of an in-service Teacher Professional Development programme be an appealing proposition? Why? Why not?","Name","School"]


        
        form.pop('front', None)
        form.pop('back', None)
        form = {new_headings[i]: form[old_heading] for i, old_heading in enumerate(form)}
        
        return form

        
        
AT = AirtableIntegration()

# Backup the data
AT.backup()  


for i in range(1, 12):
    form_path = f'Images/form_{i}/form.json'
    AT.create_records(form_path)


Images/form_1/form_1_front.jpg
Images/form_2/form_2_front.jpg
Images/form_3/form_3_front.jpg
Images/form_4/form_4_front.jpg
Images/form_5/form_5_front.jpg
Images/form_6/form_6_front.jpg
Images/form_7/form_7_front.jpg
Images/form_8/form_8_front.jpg
Images/form_9/form_9_front.jpg
Images/form_10/form_10_front.jpg
Images/form_11/form_11_front.jpg
